In [1]:
import os 
os.chdir('..')

import pandas as pd
import tellurium as te
from src.odbm.odbm_main import ModelBuilder


In [29]:
#Define DataFrames of species, reactions from input model
model_rxns = pd.read_csv('src/frenda_brenda/Files/KEGG_Filtered/Reactions_M0.csv')
model_species = pd.read_csv('src/frenda_brenda/Files/KEGG_Filtered/SpeciesBaseMechanism_biosyn.csv')

In [30]:
#Generate and save model to txt file
modelfile = 'src/frenda_brenda/Files/KEGG_Filtered/M0.txt'
myModel = ModelBuilder(model_species, model_rxns)
myModel.saveModel(modelfile)

r = []
while not r:
    try:
        r  = te.loada(myModel.compile())
    except Exception as e:
        myModel.addSpecies(str(e).split("'")[1], 0.001)
        print(str(e).split("'")[1])
        print(e)

# s = r.simulate(0,10)

In [31]:
with open('models/240828_M0.sbml', 'w') as f:
    # Write the data to the file
    f.write(r.getSBML())